In [7]:
from typing import Tuple
import tifffile
import os
from os.path import join
import numpy as np
from natsort import natsorted
from skimage.morphology import ball
from skimage.morphology import binary_erosion
from skimage.measure import regionprops
from typing import Dict, Any

def convert_instance_to_bordercore(load_dir: str, save_dir: str) -> None:
    # Load the file paths of all TIFF files in the given directory
    filepaths = load_filepaths(load_dir, extension=['tif', 'tiff', 'TIF', 'TIFF'])
    
    # Read the instance TIFF stack into an array
    instance = tifffile.imread(filepaths)
    
    # Convert the instance into the bordercore (assuming instance2border_core is already defined)
    bordercore = instance2border_core(instance, thickness=1, border_label=127, core_label=255)
    
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)
    
    # Save each slice of the bordercore as a separate TIFF file
    for i, slice in enumerate(bordercore):
        slice_filename = os.path.join(save_dir, f"slice_{i:04d}.tiff")
        tifffile.imwrite(slice_filename, slice)

def load_filepaths(load_dir: str, extension: str = None, return_path: bool = True, return_extension: bool = True) -> np.ndarray:
    """
    Given a directory path, returns an array of file paths with the specified extension.

    Args:
        load_dir: The directory containing the files.
        extension: A string or list of strings specifying the file extension(s) to search for. Optional.
        return_path: If True, file paths will include the directory path. Optional.
        return_extension: If True, file paths will include the file extension. Optional.

    Returns:
        An array of file paths.
    """
    filepaths = []
    if isinstance(extension, str):
        extension = tuple([extension])
    elif isinstance(extension, list):
        extension = tuple(extension)
    elif extension is not None and not isinstance(extension, tuple):
        raise RuntimeError("Unknown type for argument extension.")

    if extension is not None:
        extension = list(extension)
        for i in range(len(extension)):
            if extension[i][0] != ".":
                extension[i] = "." + extension[i]
        extension = tuple(extension)

    for filename in os.listdir(load_dir):
        if extension is None or str(filename).endswith(extension):
            if not return_extension:
                if extension is None:
                    filename = filename.split(".")[0]
                else:
                    for ext in extension:
                        if str(filename).endswith((ext)):
                            filename = str(filename)[:-len(ext)]
            if return_path:
                filename = join(load_dir, filename)
            filepaths.append(filename)
    filepaths = np.asarray(filepaths)
    filepaths = natsorted(filepaths)

    return filepaths

def instance2border_core(instance_seg: np.ndarray, thickness: int, border_label: int = 2, core_label: int = 1) -> np.ndarray:
    """
    Converts a label instance segmentation into a binary segmentation with border and core labels.

    Args:
        instance_seg (np.ndarray): Instance segmentation labels.
        thickness (int): The thickness of the border.
        border_label (int, optional): The label value for the border. Defaults to 2.
        core_label (int, optional): The label value for the core. Defaults to 1.

    Returns:
        np.ndarray: The converted binary segmentation labels.
    """
    instance_seg = instance_seg.astype(np.uint16)
    selem = ball(thickness, dtype=int)
    border_semantic = np.zeros_like(instance_seg, dtype=np.uint8)

    for instance in regionprops(instance_seg):
        border_semantic_particle = instance2border_core_particle(instance, instance_seg, selem, border_label, core_label, thickness + 3)
        i_start, j_start, k_start, i_end, j_end, k_end = border_semantic_particle["bbox"]
        mask = border_semantic_particle["image"]
        border_semantic[i_start:i_end, j_start:j_end, k_start:k_end][mask == border_label] = border_label
        border_semantic[i_start:i_end, j_start:j_end, k_start:k_end][mask == core_label] = core_label

    return border_semantic


def instance2border_core_particle(instance: Dict[str, Any], instance_seg: np.ndarray, selem: np.ndarray, border_label: int, core_label: int, roi_padding: int) -> Dict[str, Any]:
    """
    Converts a label instance segmentation of a particle into a binary segmentation with border and core labels.

    Args:
        instance (Dict[str, Any]): Dictionary of the particle instance properties.
        instance_seg (np.ndarray): Instance segmentation labels.
        selem (np.ndarray): Structuring element for erosion.
        border_label (int): The label value for the border.
        core_label (int): The label value for the core.
        roi_padding (int): Padding for the region of interest.

    Returns:
        Dict[str, Any]: The converted binary segmentation labels with the region of interest.
    """
    border_semantic_particle = {}
    i_start, j_start, k_start, i_end, j_end, k_end = instance["bbox"]
    # Pad the roi to improve quality of the erosion
    i_start, j_start, k_start, i_end, j_end, k_end = max(0, i_start - roi_padding), max(0, j_start - roi_padding), max(0, k_start - roi_padding), \
                                                     min(instance_seg.shape[0], i_end + roi_padding), min(instance_seg.shape[1], j_end + roi_padding), min(instance_seg.shape[2], k_end + roi_padding)
    border_semantic_particle["bbox"] = i_start, j_start, k_start, i_end, j_end, k_end
    roi_mask = instance_seg[i_start:i_end, j_start:j_end, k_start:k_end] == instance["label"]
    border_semantic_particle["image"] = roi_mask.astype(np.uint8)
    eroded = binary_erosion(roi_mask, selem)
    border_semantic_particle["image"][(eroded == 0) & (roi_mask == 1)] = border_label
    border_semantic_particle["image"][(eroded == 1) & (roi_mask == 1)] = core_label
    return border_semantic_particle

def setup_paths(dir_location, is_original_data):
    if dir_location.lower() == 'internal':
        base_path = r'C:\Senior_Design'
    elif dir_location.lower() == 'external':
        base_path = r'D:\Senior_Design'
    elif dir_location.lower() == 'cloud':
        base_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files'
    elif dir_location.lower() == 'refine':
        base_path = r'D:\Darren\Files'
    else:
        raise ValueError('Invalid directory location type')
    
    base_instance_path = os.path.join(base_path, 'database')
    if is_original_data:
        instance_path = os.path.join(base_instance_path, 'orignal_dataset', 'instance', 'tiff')
    else:
        instance_path = os.path.join(base_instance_path, 'tablet_dataset', 'instance', 'tiff')
    
    bordercore_path = instance_path.replace('instance', 'instance_to_bordercore')

    print('Paths set')
    return instance_path, bordercore_path

In [6]:
import os

instance_path, bordercore_path = setup_paths('cloud', False)
# img_names = ['2_Tablet_Aug1', '2_Tablet_Aug2', '2_Tablet_Aug3', '2_Tablet_Aug4', '2_Tablet_Aug5',
#                '4_GenericD12_Aug1', '4_GenericD12_Aug2', '4_GenericD12_Aug3', '4_GenericD12_Aug4', '4_GenericD12_Aug5',
#                '5_ClaritinD12_Aug1', '5_ClaritinD12_Aug2', '5_ClaritinD12_Aug3', '5_ClaritinD12_Aug4', '5_ClaritinD12_Aug5']
img_names = ['2_Tablet_Aug3']

for img_name in img_names:
    input_path = os.path.join(instance_path, img_name)
    output_path = os.path.join(bordercore_path, img_name)

    convert_instance_to_bordercore(input_path, output_path)
    print('Conversion for ' + img_name + ' is complete.')

Paths set
Conversion for 2_Tablet_Aug3 is complete.
